In [1]:
import uuid
import requests
from urllib.parse import urlparse
from services.knowledge_base.web_scrape import map_url, scrape_url
from services.chat.chat import llm_response
from typing import List, Dict
import openai
from humanloop import Humanloop
from dotenv import load_dotenv
import os

In [ ]:
results = await map_url("https://nogood.io/")

results


In [3]:
import uuid
import requests
from urllib.parse import urlparse
from services.knowledge_base.web_scrape import map_url, scrape_url
from services.chat.chat import llm_response
from typing import List, Dict
import openai
from humanloop import Humanloop
from dotenv import load_dotenv
import os

load_dotenv()

client = Humanloop(
    api_key=os.getenv("HUMANLOOP_API_KEY"),
)

async def create_opening_line(scraped_content: str) -> str:
    # Assuming you have OpenAI API setup
    user_prompt = f"""
    # Instructions:
    Based on this website content, create an engaging opening line for a customer service agent, like  <example_opening_line>"Welcome to WeCreate👋! Looking to boost your brand's online presence? What's your biggest goal right now?</example_opening_line>
    IMPORTANT: You must only respond with the opening line, no other text. No speech marks either, only the text.
    # Website content:
    {scraped_content[:1000]}"""
    
    system_prompt = "You are an expert at creating opening lines for customer service agents. You are given a website content and you are tasked with creating a friendly opening line for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agent_instructions(scraped_content: str) -> str:
    user_prompt = f"""
    # Instructions:
    Based on this website content, create a short sentence that begins with the name of the Company, and a short description saying what it does, like "WeCreate, a branding & digital marketing agency based Singapore, Hong Kong, and the UK."
    IMPORTANT: You must only respond with the instructions, no other text.
    # Website content:
    {scraped_content[:1000]}
    """
    
    system_prompt = "You are an expert at creating instructions for customer service agents. You are given a website content and you are tasked with creating instructions for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agents_from_urls(urls: List[str]):
    global scraped_content
    for url in urls:
        print("url:",url)
        # Get domain name for agent name
        domain = urlparse(url).netloc
        agent_name = domain.replace("www.", "")
        print("agent_name:",agent_name)
        # Map and scrape the URL
        mapped_url = await map_url(url)
        print("\n\nmapped_url:",mapped_url)
        scraped_content = await scrape_url(mapped_url[:100], user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
        web_content = ""
        for i in scraped_content:
            web_content += i['content']
        web_content = web_content[:10000]

        # Generate opening line from content
        opening_line = await create_opening_line(web_content)
        print("\n\nopening line:",opening_line)

        company_details = await create_agent_instructions(web_content)

        print("\n\ncompany details:",company_details)
        ## abstracted away lead form content, as that's determined by the user and stored in the metadata

        humanloop_prompt = client.prompts.get(
            id="pr_BiPy5GsclDI8uAzmHR8S7",
        )
        agent_instructions = humanloop_prompt.template[0]['content'].replace("{{company_details}}", company_details)

        # Create data source dict
        data_source = [{
            "id": str(uuid.uuid4()),
            "title": url,
            "data_type": "web"
        }]
        
        features = {
                    "lead_gen": {
                        "enabled": True
                    },
                    "callTransfer": {
                        "enabled": False
                    },
                    "notifyOnInterest": {
                        "enabled": False
                    },
                    "appointmentBooking": {
                        "enabled": False
                    }
                }
        
        form_fields = {
                "fields": [
                    {
                    "type": "text",
                    "label": "Full name"
                    },
                    {
                    "type": "email",
                    "label": "Email address"
                    }
                ]
            }
        
        """ POST to create new agent """
        # Prepare payload
        payload = {
            "agentName": agent_name,
            "agentPurpose": "text-chatbot-agent",
            "dataSource": data_source,
            "openingLine": opening_line,
            "language": "en-GB", 
            "voice": "Alex K",
            "voiceProvider": "cartesia",
            "instructions": agent_instructions,
            "features": features,
            "form_fields": form_fields,
            "company_name": agent_name
        }

        # POST request
        response = requests.post(
            "http://localhost:8000/api/v1/livekit/new_agent",
            json=payload,
            headers={"x-user-id": "user_2mmXezcGmjZCf88gT2v2waCBsXv"}
        )
        
        if response.status_code == 200:
            print(f"Successfully created agent for {url}")
            return response.json()
        else:
            print(f"Failed to create agent for {url}: {response.text}")



# Example usage:
urls = [
    "https://piqnic.com/"
]

response = await create_agents_from_urls(urls)
agent_id = response['data'][0]['id']

print("agent_id", agent_id)

url: https://piqnic.com/
agent_name: piqnic.com
hello, map_url


mapped_url: ['https://piqnic.com', 'https://piqnic.com/category/uncategorized', 'https://piqnic.com/author/admin', 'https://piqnic.com/the-human-firewall-the-weakest-link-of-them-all', 'https://piqnic.com/platform', 'https://piqnic.com/piqnic-partners', 'https://piqnic.com/terms-of-use', 'https://piqnic.com/book-a-free-demo', 'https://piqnic.com/book-a-free-trial', 'https://piqnic.com/contact-us', 'https://piqnic.com/cookie-policy', 'https://piqnic.com/newsletter', 'https://piqnic.com/learning', 'https://piqnic.com/solutions/use-cases/other', 'https://piqnic.com/solutions/use-cases', 'https://piqnic.com/solutions/use-cases/accounts-payable', 'https://piqnic.com/solutions/use-cases/education-enrolment', 'https://piqnic.com/solutions/use-cases/aml', 'https://piqnic.com/about-us', 'https://piqnic.com/faqs', 'https://piqnic.com/compare-plans', 'https://piqnic.com/privacy', 'https://piqnic.com/user-agreement', 'https://piqnic.

[2025-01-03 14:43:23,595][ERROR] Error processing site https://piqnic.com/category/uncategorized: can only concatenate str (not "list") to str
[2025-01-03 14:43:27,079][ERROR] Error processing site https://piqnic.com/author/admin: can only concatenate str (not "list") to str
[2025-01-03 14:43:27,962][ERROR] Error processing site https://piqnic.com: can only concatenate str (not "list") to str
[2025-01-03 14:43:35,452][ERROR] Error processing site https://piqnic.com/learning: can only concatenate str (not "list") to str
[2025-01-03 14:43:39,448][ERROR] Error processing site https://piqnic.com/cookie-policy: can only concatenate str (not "list") to str
[2025-01-03 14:43:39,451][ERROR] Error processing site https://piqnic.com/contact-us: can only concatenate str (not "list") to str
[2025-01-03 14:44:07,169][ERROR] Error processing site https://piqnic.com/about-us: can only concatenate str (not "list") to str
[2025-01-03 14:44:07,448][ERROR] Error processing site https://piqnic.com/faqs: c

hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, sliding_window_chunking
hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, sliding_window_chunking


[2025-01-03 14:44:15,579][ERROR] Error processing site https://piqnic.com/platform: can only concatenate str (not "list") to str
[2025-01-03 14:44:15,668][ERROR] Error processing site https://piqnic.com/piqnic-partners: can only concatenate str (not "list") to str


hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, sliding_window_chunking
hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, get_embedding


...

hello, get_embedding
hello, get_embedding
hello, get_embedding


[2025-01-03 14:44:17,395][ERROR] Error processing site https://piqnic.com/newsletter: can only concatenate str (not "list") to str


hello, sliding_window_chunking
hello, get_embedding
hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, insert_to_db
hello, sliding_window_chunking
hello, get_embedding
hello, insert_to_db
hello, sliding_window_chunking
hello, insert_to_db
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
he

[2025-01-03 14:44:29,823][ERROR] Error processing site https://piqnic.com/terms-of-use: can only concatenate str (not "list") to str
[2025-01-03 14:44:44,110][ERROR] Error processing site https://piqnic.com/why-piqnic: RetryError[<Future at 0x12bd2fbf0 state=finished raised HTTPError>]
[2025-01-03 14:44:44,113][ERROR] Error processing site https://learning.piqnic.com/administration/functional-model: RetryError[<Future at 0x12bb9c440 state=finished raised HTTPError>]
[2025-01-03 14:44:44,115][ERROR] Error processing site https://blog.piqnic.com/blog/introducing-piqnic-2.0-unifying-work-management-to-end-chaos: RetryError[<Future at 0x12bb9c410 state=finished raised HTTPError>]
[2025-01-03 14:44:44,153][ERROR] Error processing site https://piqnic.com/solutions/use-cases/insurance-claims: RetryError[<Future at 0x11f50e7b0 state=finished raised HTTPError>]
[2025-01-03 14:44:44,192][ERROR] Error processing site https://piqnic.com/solutions/use-cases/accounts-payable: RetryError[<Future at 0

end, scrape_url


opening line: Welcome to Piqnic! We're excited to help you with document management solutions. What type of documents are you looking to organize today?


company details: Piqnic, a document management solutions provider offering services across multiple countries.
Successfully created agent for https://piqnic.com/
agent_id f4944c71-e707-421b-817c-af9206be1a4a


In [2]:
""" CHAT WITH AGENT"""
from services.chat.lk_chat import lk_chat_process

async def test_chat(message: str , agent_id: str):
    room_name = "test_room_45iiiij6ii"

    try:
        # Get the response generator
        response_generator = lk_chat_process(message, agent_id, room_name)
        
        print("\n\nAgent response:\n\n")
        # Stream the responses
        async for chunk in response_generator:
            # In a real application, you might want to send this to a websocket or other stream
            print('\033[96m' + chunk + '\033[0m', end='', flush=True)   

    except Exception as e:
        print(f"Error occurred: {str(e)}")

await test_chat(
    message="yes, we need an MVP launch", 
    agent_id="55476322-d82a-4815-8fda-a57beecb933c")



Agent response:


Launching an MVP is an exciting step! Could you share what specific marketing goals you're looking to achieve with this launch over the next six months? This will help us tailor our insights and strategies to best fit your needs.

### Recall Meeting bots

In [ ]:
import requests

# API endpoint
url = "https://us-west-2.recall.ai/api/v1/bot/"

# Your authorization token
token = "209a894215526cd61d70ed2f77df6cb39f6b0e4d"

# Request headers
headers = {
    "Authorization": f"Token {token}",
    "Content-Type": "application/json"
}

# Request payload
payload = {
    "meeting_url": "https://meet.google.com/stn-gobs-xcr",
    "bot_name": "Bot"
}

# Send POST request
response = requests.post(url, headers=headers, json=payload)

# Check response
print(response.status_code)
print(response.json())

In [ ]:
bot_joining = response.json()

bot_joining

In [ ]:
import requests

url = "https://us-west-2.recall.ai/api/v1/bot/e09e55e2-a067-473d-80be-18b9725ba7b8/"

headers = {
    "accept": "application/json",
    "Authorization": "209a894215526cd61d70ed2f77df6cb39f6b0e4d"
}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
response.json()